In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('train-data.csv', sep=';')
df.drop_duplicates(inplace=True)

GSCPI_df = pd.read_csv('GSCPI_data.csv')
GSCPI_df[['Year','Month']] = GSCPI_df['Year-Month'].str.split('-',expand=True)
GSCPI_df['Month'] = GSCPI_df['Month'].astype(int)
GSCPI_df['Year'] = GSCPI_df['Year'].astype(int)
GSCPI_df.drop(columns=['Year-Month'], inplace=True)

LPI_df = pd.read_csv('LPIextend.csv')

worldbank_economic_data_df = pd.read_csv('worldbank_economic_data.csv')

worldbank_inflation_data_df = pd.read_csv('worldbank_inflation_data.csv')
worldbank_inflation_data_df[['Year','Month']] = worldbank_inflation_data_df['Year-Month'].str.split('-',expand=True)
worldbank_inflation_data_df['Year'] = worldbank_inflation_data_df['Year'].astype(int)
worldbank_inflation_data_df['Month'] = worldbank_inflation_data_df['Month'].astype(int) 
worldbank_inflation_data_df.drop(columns=['Year-Month'], inplace=True)

In [4]:
df['Reference proxy'] = df['Reference proxy'].apply(lambda x: x.replace('reference-', ''))
df['Product  Line proxy'] = df['Product  Line proxy'].apply(lambda x: x.replace('Product Line-', ''))
df['Division proxy'] = df['Division proxy'].apply(lambda x: x.replace('Division-', ''))
df['Customer Persona proxy'] = df['Customer Persona proxy'].apply(lambda x: x.replace('Customer Segmentation-', ''))
df['Strategic Product Family proxy'] = df['Strategic Product Family proxy'].apply(lambda x: x.replace('Strategic Product Family-', ''))

(1762970, 19)

In [6]:
country_mapping = {'DE': 'Germany', 'CN': 'China', 'GB': 'United Kingdom', 'AU': 'Australia', 'ES': 'Spain', 'NL': 'Netherlands', 'US': 'United States', 'DK': 'Denmark', 'BE': 'Belgium', 'FR': 'France', 'IT': 'Italy', 'PL': 'Poland', 'SE': 'Sweden', 'TW': 'Taiwan', 'JP': 'Japan', 'HK': 'Hong Kong', 'KR': 'South Korea', 'PH': 'Philippines', 'MY': 'Malaysia', 'SG': 'Singapore', 'TH': 'Thailand', 'Id': 'Indonesia', 'FJ': 'Fiji', 'VN': 'Vietnam', 'BN': 'Brunei', 'NZ': 'New Zealand', 'MM': 'Myanmar', 'KH': 'Cambodia', 'MN': 'Mongolia', 'LA': 'Laos', 'PG': 'Papua New Guinea'}
df['Country'] = df['Country'].map(country_mapping)


In [7]:
df['Reference proxy'] = df['Reference proxy'].apply(lambda df: df.replace('reference-', ''))
df['Product  Line proxy'] = df['Product  Line proxy'].apply(lambda df: df.replace('Product Line-', ''))
df['Division proxy'] = df['Division proxy'].apply(lambda df: df.replace('Division-', ''))
df['Customer Persona proxy'] = df['Customer Persona proxy'].apply(lambda df: df.replace('Customer Segmentation-', ''))
df['Strategic Product Family proxy'] = df['Strategic Product Family proxy'].apply(lambda df: df.replace('Strategic Product Family-', ''))

df[['Quarter', 'Year']] = df.Date.str.split(expand=True)
df['Quarter'] = df['Quarter'].map({'jan-apr': 1, 'may-jul': 2, 'may-aug': 2, 'sep-dec': 3})
df.drop('Date', axis=1, inplace=True)

df_melted = pd.melt(df[['Month 1', 'Month 2', 'Month 3', 'Month 4']], var_name='Month', value_name='Sales', ignore_index=False)
df_melted['Month'] = df_melted['Month'].str.replace('Month ', '').astype(int)
df = df.merge(df_melted, left_index=True, right_index=True)
df['Month'] = df['Month'] + 4*(df['Quarter']-1)
df.drop(['Month 1', 'Month 2', 'Month 3', 'Month 4', 'Quarter'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

df['Year'] = df['Year'].astype(int)

In [8]:
df.shape

(7051880, 17)

In [9]:
df = df.merge(LPI_df, on='Country')
df.shape

(7051880, 42)

In [10]:
df = df.merge(worldbank_economic_data_df, how='left', on=['Country', 'Year'])
df.shape

(7051880, 51)

In [11]:
df = df.merge(worldbank_inflation_data_df, how='left', on=['Country', 'Year', 'Month'])
df.shape

(7051880, 53)

In [12]:
df = df.merge(GSCPI_df, how='left', on=['Year','Month'])
df.shape

(7051880, 54)

In [13]:
df['Month_number'] = df['Month'] + (df['Year']-2020)*12-8

In [14]:
df = df.reindex(columns=['index', 'Year','Month','Month_number','id_product', 'Region', 'Country','Site','Operations','Zone','Cluster','Reference proxy','Product  Line proxy','Division proxy','Customer Persona proxy','Strategic Product Family proxy','Product Life cycel status','Sales','Unnamed: 0','ID','population (2023)','area','landAreaKm','unMember','netChange','growthRate','worldPercentage','density','densityMi','rank','LPI Grouped Rank','Customs Score','Customs Grouped Rank','Infrastructure Score','Infrastructure Grouped Rank','International Shipments Score','International Shipments Grouped Rank','Logistics Competence and Quality Score','Logistics Competence and Quality Grouped Rank','Timeliness Score','Timeliness Grouped Rank','Tracking and Tracing Score','Tracking and Tracing Grouped Rank','Agriculture, forestry, and fishing, value added (annual % growth)','Exports of goods and services (annual % growth)','Final consumption expenditure (annual % growth)','GDP (current US$)','Gross capital formation (annual % growth)','Imports of goods and services (annual % growth)','Industry (including construction), value added (annual % growth)','Manufacturing, value added (annual % growth)','Services, value added (annual % growth)','Energy Price Index','Headline Consumer Price Index','GSCPI','Sales'])